In [1]:
import pandas as pd
import numpy as np

## Data Loading: read the csv file

In [3]:
# Use Windows-1252 Encoding to read the file(used for files with non-UTF-8 characters).
df = pd.read_csv('laptop_price.csv', encoding='cp1252')
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [5]:
df.shape

(1303, 13)

In [7]:
df.duplicated().sum()

0

In [9]:
df.isnull().sum()

laptop_ID           0
Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price_euros         0
dtype: int64

### Perform a copy of (pf = df): Save to a new CSV file

In [11]:
# Perform processing (save the new csv file after all of the feature engineering)
pf = df.copy()

In [13]:
# Data validation: change the data types
pf['Ram'] = pf['Ram'].str.replace('GB', '').astype('int64')
pf['Weight'] = pf['Weight'].str.replace('kg', '').astype('float32')

pf['Ram'] = pf['Ram'].astype('int64')
pf[['Weight','Price_euros']] = pf[['Weight','Price_euros']].astype('float32')

In [15]:
pf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   Product           1303 non-null   object 
 3   TypeName          1303 non-null   object 
 4   Inches            1303 non-null   float64
 5   ScreenResolution  1303 non-null   object 
 6   Cpu               1303 non-null   object 
 7   Ram               1303 non-null   int64  
 8   Memory            1303 non-null   object 
 9   Gpu               1303 non-null   object 
 10  OpSys             1303 non-null   object 
 11  Weight            1303 non-null   float32
 12  Price_euros       1303 non-null   float32
dtypes: float32(2), float64(1), int64(2), object(8)
memory usage: 122.3+ KB


# Exploratory Data Analysis (EDA)

## Data Transformation

### Insert Two columns with TouchScreen and IPS

In [19]:
pf['TouchScreen'] = pf['ScreenResolution'].apply(lambda x:1 if 'Touchscreen' in x else 0)
pf['IPS'] = pf['ScreenResolution'].apply(lambda x:1 if 'IPS' in x else 0)
pf.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,TouchScreen,IPS
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.689941,0,1
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.940002,0,0
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.000000,0,0
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.449951,0,1
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.599976,0,1


### Add PPI column with x_resolution and y_resolution columns

In [21]:
# Add the x_resolution and y_resolution columns first and change the types to integer
pf['x_resolution'] = pf['ScreenResolution'].apply(lambda x: x[x.index('x') - 4 : x.index('x')])
pf['y_resolution'] = pf['ScreenResolution'].apply(lambda x:x.split('x')[-1])
pf['x_resolution'] = pf['x_resolution'].astype(int)
pf['y_resolution'] = pf['y_resolution'].astype(int)

In [23]:
# check about the correlation for x_resolution and y_resolution to the price
correlation = pf[['x_resolution','y_resolution','Price_euros']].corr()['Price_euros']
correlation

x_resolution    0.556529
y_resolution    0.552809
Price_euros     1.000000
Name: Price_euros, dtype: float64

In [25]:
# Merge x_resolution and y_resolution to PPI because there are highly correlation.
pf['PPI'] = (pow(pf['x_resolution']**2 + pf['y_resolution']**2, 0.5)) / pf['Inches'].astype(float)
pf.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,TouchScreen,IPS,x_resolution,y_resolution,PPI
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.689941,0,1,2560,1600,226.983005
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.940002,0,0,1440,900,127.677940
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.000000,0,0,1920,1080,141.211998
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.449951,0,1,2880,1800,220.534624
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.599976,0,1,2560,1600,226.983005


In [27]:
# check the correlation for the PPI to the price
correlation = pf[['PPI','Price_euros']].corr()['Price_euros']
correlation

PPI            0.473487
Price_euros    1.000000
Name: Price_euros, dtype: float64

In [29]:
# Drop the Columns x_resolution, y_resolution, ScreenResolution
pf = pf.drop(columns=['laptop_ID','x_resolution','y_resolution','ScreenResolution'])
pf.head()

,Company,Product,TypeName,Inches,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,TouchScreen,IPS,PPI
0,Apple,MacBook Pro,Ultrabook,13.3,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.689941,0,1,226.983005
1,Apple,Macbook Air,Ultrabook,13.3,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.940002,0,0,127.677940
2,HP,250 G6,Notebook,15.6,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.000000,0,0,141.211998
3,Apple,MacBook Pro,Ultrabook,15.4,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.449951,0,1,220.534624
4,Apple,MacBook Pro,Ultrabook,13.3,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.599976,0,1,226.983005


### Add CPU Brand columns

In [31]:
pf['Cpu'].value_counts()

Cpu
Intel Core i5 7200U 2.5GHz       190
Intel Core i7 7700HQ 2.8GHz      146
Intel Core i7 7500U 2.7GHz       134
Intel Core i7 8550U 1.8GHz        73
Intel Core i5 8250U 1.6GHz        72
                                ... 
Intel Core M M3-6Y30 0.9GHz        1
AMD A9-Series 9420 2.9GHz          1
Intel Core i3 6006U 2.2GHz         1
AMD A6-Series 7310 2GHz            1
Intel Xeon E3-1535M v6 3.1GHz      1
Name: count, Length: 118, dtype: int64

In [33]:
# Change the Cpu system name with first 3 words
pf['Cpu'] = pf['Cpu'].apply(lambda x:' '.join(x.split()[:3]))
pf['Cpu'].value_counts()

Cpu
Intel Core i7               527
Intel Core i5               423
Intel Core i3               136
Intel Celeron Dual           80
Intel Pentium Quad           27
Intel Core M                 19
AMD A9-Series 9420           12
Intel Celeron Quad            8
AMD A6-Series 9220            8
AMD A12-Series 9720P          7
Intel Atom x5-Z8350           5
AMD A8-Series 7410            4
Intel Atom x5-Z8550           4
Intel Pentium Dual            3
AMD A9-Series 9410            3
AMD Ryzen 1700                3
AMD A9-Series A9-9420         2
AMD A10-Series 9620P          2
Intel Atom X5-Z8350           2
AMD E-Series E2-9000e         2
Intel Xeon E3-1535M           2
Intel Xeon E3-1505M           2
AMD E-Series 7110             2
AMD A10-Series 9600P          2
AMD A6-Series A6-9220         2
AMD A10-Series A10-9620P      2
AMD Ryzen 1600                1
Intel Atom x5-Z8300           1
AMD E-Series E2-6110          1
AMD FX 9830P                  1
AMD E-Series E2-9000          1
AMD 

In [35]:
# Add new column: CPU Brand
def fetch_processor(cpu):
    if cpu in {'Intel Core i7','Intel Core i5','Intel Core i3'}:
        return cpu
    elif 'Intel' in cpu:
        return 'Other Intel Processor'
    return 'AMD Processor'

pf['CPU Brand'] = pf['Cpu'].apply(fetch_processor)
pf = pf.drop(columns=['Cpu'])
pf.head()

,Company,Product,TypeName,Inches,Ram,Memory,Gpu,OpSys,Weight,Price_euros,TouchScreen,IPS,PPI,CPU Brand
0,Apple,MacBook Pro,Ultrabook,13.3,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.689941,0,1,226.983005,Intel Core i5
1,Apple,Macbook Air,Ultrabook,13.3,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.940002,0,0,127.677940,Intel Core i5
2,HP,250 G6,Notebook,15.6,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.000000,0,0,141.211998,Intel Core i5
3,Apple,MacBook Pro,Ultrabook,15.4,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.449951,0,1,220.534624,Intel Core i7
4,Apple,MacBook Pro,Ultrabook,13.3,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.599976,0,1,226.983005,Intel Core i5


### Add GPU Brand column

In [38]:
pf['Gpu'].value_counts()

Gpu
Intel HD Graphics 620      281
Intel HD Graphics 520      185
Intel UHD Graphics 620      68
Nvidia GeForce GTX 1050     66
Nvidia GeForce GTX 1060     48
                          ... 
AMD Radeon R5 520            1
AMD Radeon R7                1
Intel HD Graphics 540        1
AMD Radeon 540               1
ARM Mali T860 MP4            1
Name: count, Length: 110, dtype: int64

In [40]:
# change the Gpu name with the first word
pf['Gpu'] = pf['Gpu'].apply(lambda x:x.split()[0])
pf['Gpu'].value_counts()

Gpu
Intel     722
Nvidia    400
AMD       180
ARM         1
Name: count, dtype: int64

In [42]:
# Delete the row with Gpu = ARM
pf = pf[pf['Gpu'] != 'ARM']

In [44]:
# rename the column name of Gpu to GPU Brand
pf = pf.rename(columns={'Gpu':'GPU Brand'})
pf.head()

,Company,Product,TypeName,Inches,Ram,Memory,GPU Brand,OpSys,Weight,Price_euros,TouchScreen,IPS,PPI,CPU Brand
0,Apple,MacBook Pro,Ultrabook,13.3,8,128GB SSD,Intel,macOS,1.37,1339.689941,0,1,226.983005,Intel Core i5
1,Apple,Macbook Air,Ultrabook,13.3,8,128GB Flash Storage,Intel,macOS,1.34,898.940002,0,0,127.677940,Intel Core i5
2,HP,250 G6,Notebook,15.6,8,256GB SSD,Intel,No OS,1.86,575.000000,0,0,141.211998,Intel Core i5
3,Apple,MacBook Pro,Ultrabook,15.4,16,512GB SSD,AMD,macOS,1.83,2537.449951,0,1,220.534624,Intel Core i7
4,Apple,MacBook Pro,Ultrabook,13.3,8,256GB SSD,Intel,macOS,1.37,1803.599976,0,1,226.983005,Intel Core i5


### Add OS columns

In [46]:
pf['OpSys'].value_counts()

OpSys
Windows 10      1072
No OS             66
Linux             62
Windows 7         45
Chrome OS         26
macOS             13
Mac OS X           8
Windows 10 S       8
Android            2
Name: count, dtype: int64

In [48]:
# Change the catogory into Windows, Mac and Others/No OS/Linux
def cat_OS(inp):
    if inp in {'Windows 10','Windows 7','Windows 10 S'}:
        return 'Windows'
    elif inp in {'Mac OC X','macOS'}:
        return 'Mac'
    else:
        return 'Others/No OS/Linux'

pf['OpSys'] = pf['OpSys'].apply(cat_OS)
pf = pf.rename(columns={'OpSys':'OS'})
pf.head()

,Company,Product,TypeName,Inches,Ram,Memory,GPU Brand,OS,Weight,Price_euros,TouchScreen,IPS,PPI,CPU Brand
0,Apple,MacBook Pro,Ultrabook,13.3,8,128GB SSD,Intel,Mac,1.37,1339.689941,0,1,226.983005,Intel Core i5
1,Apple,Macbook Air,Ultrabook,13.3,8,128GB Flash Storage,Intel,Mac,1.34,898.940002,0,0,127.677940,Intel Core i5
2,HP,250 G6,Notebook,15.6,8,256GB SSD,Intel,Others/No OS/Linux,1.86,575.000000,0,0,141.211998,Intel Core i5
3,Apple,MacBook Pro,Ultrabook,15.4,16,512GB SSD,AMD,Mac,1.83,2537.449951,0,1,220.534624,Intel Core i7
4,Apple,MacBook Pro,Ultrabook,13.3,8,256GB SSD,Intel,Mac,1.37,1803.599976,0,1,226.983005,Intel Core i5


In [50]:
pf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 0 to 1302
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1302 non-null   object 
 1   Product      1302 non-null   object 
 2   TypeName     1302 non-null   object 
 3   Inches       1302 non-null   float64
 4   Ram          1302 non-null   int64  
 5   Memory       1302 non-null   object 
 6   GPU Brand    1302 non-null   object 
 7   OS           1302 non-null   object 
 8   Weight       1302 non-null   float32
 9   Price_euros  1302 non-null   float32
 10  TouchScreen  1302 non-null   int64  
 11  IPS          1302 non-null   int64  
 12  PPI          1302 non-null   float64
 13  CPU Brand    1302 non-null   object 
dtypes: float32(2), float64(2), int64(3), object(7)
memory usage: 142.4+ KB


### Descriptive Statistics (Mean, Median and Mode)

In [53]:
mean_values = pf.mean(numeric_only=True)
print("Mean:\n", mean_values)
print()

median_values = pf.median(numeric_only=True)
print("Median:\n", median_values)
print()

mode_values = pf.mode().iloc[0]  # `.iloc[0]` to get the first mode if there are multiple
print("Mode:\n", mode_values)

Mean:
 Inches           15.019278
Ram               8.385561
Weight            2.039416
Price_euros    1124.043823
TouchScreen       0.146697
IPS               0.279570
PPI             146.568497
dtype: float64

Median:
 Inches          15.600000
Ram              8.000000
Weight           2.040000
Price_euros    978.000000
TouchScreen      0.000000
IPS              0.000000
PPI            141.211998
dtype: float64

Mode:
 Company                 Dell
Product               XPS 13
TypeName            Notebook
Inches                  15.6
Ram                      8.0
Memory             256GB SSD
GPU Brand              Intel
OS                   Windows
Weight                   2.2
Price_euros           1099.0
TouchScreen              0.0
IPS                      0.0
PPI               141.211998
CPU Brand      Intel Core i7
Name: 0, dtype: object


### Correlation Analysis: Check the Correlation with the Price_euros

In [55]:
numerical_features = pf.select_dtypes(include=['number']).columns
numerical_pf = pf[numerical_features]

correlation = numerical_pf.corr()['Price_euros']
correlation

Inches         0.067329
Ram            0.742905
Weight         0.209867
Price_euros    1.000000
TouchScreen    0.192917
IPS            0.253320
PPI            0.475368
Name: Price_euros, dtype: float64

## Features Specific Analysis: Overall Metrics and Calculation

In [57]:
#Key Matrics No.1,2 and 3
Total_Amount = pf['Price_euros'].sum()
Total_Quantity = pf.shape[0]
Average_Amount_per_Quantity = Total_Amount / Total_Quantity
print(f'Total Amount:{Total_Amount}')
print(f'Total Quantity:{Total_Quantity}')
print(f'Average_Amount_per_Quantity:{Average_Amount_per_Quantity}')

Total Amount:1463505.125
Total Quantity:1302
Average_Amount_per_Quantity:1124.0438748079878


In [59]:
# No.4 Total Amount, Total Quantity and Average Amount per Quantity by Company, RAM
grouped_pf_full = pf.groupby(['Company', 'Ram']).agg(
    Total_Amount=('Price_euros', 'sum'),
    Total_Quantity=('Price_euros', 'count'),
    Average_Amount_per_Quantity=('Price_euros', 'mean')
).reset_index()
grouped_pf_sorted = grouped_pf_full.sort_values(by='Average_Amount_per_Quantity', ascending=False).reset_index(drop=True)
grouped_pf_sorted.head()

,Company,Ram,Total_Amount,Total_Quantity,Average_Amount_per_Quantity
0,Razer,32,11598.000000,2,5799.000000
1,Asus,64,3975.000000,1,3975.000000
2,Lenovo,32,10802.000000,3,3600.666748
3,Dell,32,22056.460938,7,3150.923096
4,MSI,32,2799.000000,1,2799.000000


In [61]:
# No.5, 6 and 7: Total Amount and Total Quantity by Inches, Touchscreen and IPS
## Define a function to calculate the Total Amount and Total Quantity
def calculate_figure(pf, group_column):
    grouped_pf = pf.groupby(group_column).agg(
        Total_Amount=('Price_euros','sum'),
        Total_Quantity=('Price_euros','count')
    ).reset_index()

    # Sorting by Total_Amount in descending order
    grouped_pf = grouped_pf.sort_values(by='Total_Amount', ascending=False).reset_index(drop=True)
    
    # Formatting the 'Percentage_Total_Amount' and 'Percentage_Total_Quantity' to two decimal places
    grouped_pf['Total_Amount'] = grouped_pf['Total_Amount'].map('{:,.2f}'.format)

    return grouped_pf

#Generate Table for each features
figure_by_inches = calculate_figure(pf, 'Inches')
figure_by_Touchscreen = calculate_figure(pf, 'TouchScreen')
figure_by_IPS = calculate_figure(pf, 'IPS')

# Display all tables in a format similar to the one you provided
print("Total Amount and Total Quantity by Inches:")
display(figure_by_inches.head())

print("\nTotal Amount and Total Quantity by Touchscreen:")
display(figure_by_Touchscreen)

print("Total Amount and Total Quantity by IPS:")
display(figure_by_IPS)

Total Amount and Total Quantity by Inches:


,Inches,Total_Amount,Total_Quantity
0,15.6,"600,731.56",665
1,17.3,"274,382.06",164
2,14.0,"236,847.05",197
3,13.3,"218,500.20",164
4,12.5,"61,756.23",39



Total Amount and Total Quantity by Touchscreen:


,TouchScreen,Total_Amount,Total_Quantity
0,0,"1,186,704.00",1111
1,1,"276,801.19",191


Total Amount and Total Quantity by IPS:


,IPS,Total_Amount,Total_Quantity
0,0,"950,903.31",938
1,1,"512,601.81",364


In [63]:
# No. 8, 9, 10 and 11: Percentage of Total Amount and Total Quantity by TypeName, CPU Brand, GPU Brand and OS
## Define a function to calculate Total Amount, Total Quantity, and their percentages
def calculate_percentage(pf, group_column):
    # Grouping by the specified column to calculate Total Amount and Total Quantity
    grouped_pf = pf.groupby(group_column).agg(
        Total_Amount=('Price_euros', 'sum'),
        Total_Quantity=('Price_euros', 'count')
    ).reset_index()
    
    # Calculating the percentage of Total Amount and Total Quantity
    total_amount_sum = grouped_pf['Total_Amount'].sum()
    total_quantity_sum = grouped_pf['Total_Quantity'].sum()
    
    grouped_pf['Percentage_Total_Amount'] = (grouped_pf['Total_Amount'] / total_amount_sum) * 100
    grouped_pf['Percentage_Total_Quantity'] = (grouped_pf['Total_Quantity'] / total_quantity_sum) * 100
    
    # Sorting by Percentage_Total_Amount in descending order
    grouped_pf = grouped_pf.sort_values(by='Percentage_Total_Amount', ascending=False).reset_index(drop=True)
    
    # Formatting the 'Percentage_Total_Amount' and 'Percentage_Total_Quantity' to two decimal places
    grouped_pf['Percentage_Total_Amount'] = grouped_pf['Percentage_Total_Amount'].map('{:.2f}%'.format)
    grouped_pf['Percentage_Total_Quantity'] = grouped_pf['Percentage_Total_Quantity'].map('{:.2f}%'.format)
    
    return grouped_pf

# Generate tables for each feature
percentage_by_typename = calculate_percentage(pf, 'TypeName')
percentage_by_cpu = calculate_percentage(pf, 'CPU Brand')
percentage_by_gpu = calculate_percentage(pf, 'GPU Brand')
percentage_by_os = calculate_percentage(pf, 'OS')

# Display all tables in a format similar to the one you provided
print("Percentage of Total Amount and Total Quantity by TypeName:")
display(percentage_by_typename)

print("\nPercentage of Total Amount and Total Quantity by CPU Brand:")
display(percentage_by_cpu)

print("\nPercentage of Total Amount and Total Quantity by GPU Brand:")
display(percentage_by_gpu)

print("\nPercentage of Total Amount and Total Quantity by OS:")
display(percentage_by_os)

Percentage of Total Amount and Total Quantity by TypeName:


,TypeName,Total_Amount,Total_Quantity,Percentage_Total_Amount,Percentage_Total_Quantity
0,Notebook,568571.187500,727,38.85%,55.84%
1,Gaming,354933.031250,205,24.25%,15.75%
2,Ultrabook,303453.156250,196,20.73%,15.05%
3,2 in 1 Convertible,154511.593750,120,10.56%,9.22%
4,Workstation,66130.406250,29,4.52%,2.23%
5,Netbook,15905.799805,25,1.09%,1.92%



Percentage of Total Amount and Total Quantity by CPU Brand:


,CPU Brand,Total_Amount,Total_Quantity,Percentage_Total_Amount,Percentage_Total_Quantity
0,Intel Core i7,840975.562500,527,57.46%,40.48%
1,Intel Core i5,429351.437500,423,29.34%,32.49%
2,Other Intel Processor,84757.367188,154,5.79%,11.83%
3,Intel Core i3,73661.132812,136,5.03%,10.45%
4,AMD Processor,34759.609375,62,2.38%,4.76%



Percentage of Total Amount and Total Quantity by GPU Brand:


,GPU Brand,Total_Amount,Total_Quantity,Percentage_Total_Amount,Percentage_Total_Quantity
0,Intel,727938.875000,722,49.74%,55.45%
1,Nvidia,595949.687500,400,40.72%,30.72%
2,AMD,139616.578125,180,9.54%,13.82%



Percentage of Total Amount and Total Quantity by OS:


,OS,Total_Amount,Total_Quantity,Percentage_Total_Amount,Percentage_Total_Quantity
0,Windows,1.338437e+06,1125,91.45%,86.41%
1,Others/No OS/Linux,1.023233e+05,164,6.99%,12.60%
2,Mac,2.274520e+04,13,1.55%,1.00%


In [65]:
# Save the new dataset to new csv file
pf.to_csv("new_laptop_price.csv", index=False)